In [1]:
import glob

import xarray

In [2]:
# Download from S3 - specify path here.

NC_FILE_PATH   = "<path_to_files>/netcdf4/"
ZARR_FILE_PATH = "<path_to_files>/zarr/"

In [3]:
%time xarr = xarray.open_mfdataset(NC_FILE_PATH, combine='by_coords').load()

CPU times: user 3min 18s, sys: 13.9 s, total: 3min 31s
Wall time: 3min 45s


## Generating zarr files

Initally, we needed to generate the files in a zarr format. `xarray` can currently only read zarr files which it has itself written, using:

```python
xarr.to_zarr(ZARR_FILE_PATH, consolidated=True)
```

Since the files are already contained in the S3 bucket, we can go ahead and use those directly.

In [5]:
%time xarray.open_zarr(ZARR_FILE_PATH).load()

CPU times: user 45.4 s, sys: 6.69 s, total: 52.1 s
Wall time: 17.4 s


<xarray.Dataset>
Dimensions:    (latitude: 169, level: 1, longitude: 293, time: 87600)
Coordinates:
  * latitude   (latitude) float64 74.0 73.75 73.5 73.25 ... 32.5 32.25 32.0
  * level      (level) float64 1e+03
  * longitude  (longitude) float64 333.0 333.2 333.5 333.8 ... 45.5 45.75 46.0
  * time       (time) datetime64[ns] 1979-01-01 ... 1989-01-02T23:00:00
Data variables:
    R          (time, level, latitude, longitude) float32 44.83952 ... 83.39159
Attributes:
    CONVERSION_DATE:      Mon May 13 18:31:49 MDT 2019
    CONVERSION_PLATFORM:  Linux casper13 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    Conventions:          CF-1.6
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NCO:                  netCDF Operators version 4.7.9 (Homepage = http://n...
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB 1 data to netC...
    NETCDF_VERSION:       4.6.1
    history:              Mon Mar 30 02:31:12 2020: ncks -d latitude,32.0,74....

## `dask.distributed`

The above reading took ~ 50s. We can improve this by creating a local dask cluster, to which `xarray` then delegates scheduling the tasks for reading files from disk.

This has the advatanges of being faster and very easily scalable. Were we to run this across multiple nodes in a cloud-based cluster, we'd need only specify a different scheduler - the API is otherwise unchanged.

In [2]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()  # Defaults to 16 workers
client = Client(cluster)

xarr = xarray.open_zarr("/home/tom/tmp/relative-humidity-data/zarr", consolidated=True)

In [3]:
%%prun -q -T prun_output.txt

loaded_xarr = xarr.load()

 
*** Profile printout saved to text file 'prun_output.txt'. 


## Profiling

This takes anywhere from 30 - 40s on our machines (usually depending on what other activity is present).

We can see below that a lot of this time is spend by various threads awaiting access to some lock.

In [5]:
print(open("prun_output.txt").read())

         671327 function calls (537788 primitive calls) in 37.911 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12   24.890    2.074   24.890    2.074 {method 'acquire' of '_thread.lock' objects}
        1   10.846   10.846   10.891   10.891 core.py:4318(concatenate3)
        1    1.235    1.235   37.911   37.911 dataset.py:625(load)
     4098    0.122    0.000    0.134    0.000 base_events.py:738(_call_soon)
        5    0.105    0.021    0.108    0.022 core.py:263(reverse_dict)
    36868    0.077    0.000    0.077    0.000 utils.py:751(tokey)
36866/4097    0.045    0.000    0.071    0.000 utils_comm.py:166(unpack_remotedata)
     8197    0.044    0.000    0.048    0.000 core.py:159(get_dependencies)
     4097    0.043    0.000    0.043    0.000 {built-in method _pickle.dumps}
91274/28679    0.042    0.000    0.051    0.000 core.py:234(flatten)
        2    0.033    0.017    0.059    0.029 {built-in method builtin